In [ ]:
import mclimate as mclim
import farray as fa
import importlib
import numpy as np
import bottleneck
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colors as mc
importlib.reload(mc)

In [ ]:
date = '2016012100'
plt.style.use('seaborn-colorblind')

In [ ]:
gefs_mean = xr.open_dataset('../../espr/reforecast/v12/slp_mean_djf.nc',chunks={'time':10})
gefs_mean = gefs_mean.sel(date=date).sel(time=54)
gefs_mean

In [ ]:
gefs_sprd = xr.open_dataset('../../espr/reforecast/v12/slp_sprd_djf.nc',chunks={'time':10})
gefs_sprd = gefs_sprd.sel(date=date).sel(time=54)
gefs_sprd

In [ ]:
mclimate = mclim.MClimate('2016-01-21','G:/Projects/espr/reforecast/v12','slp',54,v12=True,dask_enabled=True)

In [ ]:
mclimate.client

In [ ]:
mcli = mclimate.generate()

In [ ]:
mcli_sprd = mclimate.generate(stat='sprd')

In [ ]:
# percentiles = percentile(mc_mu, gefs_mean)
# subset = subset_sprd(percentiles, mc_std)
# hsa_final = hsa_transform(gefs_sprd, subset)

In [ ]:
percentile = mcli.load().rank('time')/len(mcli['time'])

In [ ]:
new_perc = percentile.where(np.logical_and(percentile >= percentile.isel(time=-1)-0.05, percentile <= percentile.isel(time=-1)+0.05),drop=True)

In [ ]:
mcli_sprd = mcli_sprd.where(~np.isnan(new_perc),drop=True)

In [ ]:
mcli_sprd = mcli_sprd.chunk({'time':10})

In [ ]:
mcli_sprd

In [ ]:
subset_vals = (gefs_sprd - mcli_sprd.mean('time'))/mcli_sprd.std('time')
subset_vals = (0.99-(-0.99))*(subset_vals-subset_vals.min(['latitude','longitude']))/(subset_vals.max(['latitude','longitude'])-subset_vals.min(['latitude','longitude'])) + -0.99
subset_vals = np.arctanh(subset_vals)

In [ ]:
subset_vals

In [ ]:
subset_vals.where(np.abs(subset_vals) > 0.5).msl

In [ ]:
n=35
x = 0.5
cmap = plt.cm.RdBu_r
lower = cmap(np.linspace(0, x, n))
white = np.ones((80-2*n,4))
upper = cmap(np.linspace(1-x, 1, n))
colors = np.vstack((lower, white, upper))
tmap = mc.LinearSegmentedColormap.from_list('map_white', colors)

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection=ccrs.Miller())
ax.add_feature(cf.NaturalEarthFeature(
    'cultural', 'admin_1_states_provinces_lines', '50m',
    edgecolor='gray', facecolor='none'))
ax.add_feature(cf.NaturalEarthFeature(
    'cultural', 'admin_1_states_provinces_lines', '50m',
    edgecolor='gray', facecolor='none'))        
ax.add_feature(cf.LAKES, facecolor='gray')
ax.add_feature(cf.BORDERS, edgecolor='gray')
ax.add_feature(cf.RIVERS, edgecolor='gray')
ax.add_feature(cf.OCEAN, facecolor='gray')
ax.set_extent([-180,-50,20,60],crs=ccrs.PlateCarree())
ax.coastlines(resolution='50m')
subset_vals_fix = subset_vals
subset_vals_fix['longitude'] = (subset_vals_fix['longitude']  + 180) % 360 - 180
subset_vals_fix  = subset_vals_fix.sortby(subset_vals_fix['longitude'] )
c = subset_vals_fix.msl.plot.imshow(ax=ax,
                                     transform=ccrs.PlateCarree(),
                                    cmap=tmap,
                                    add_colorbar=False)
(gefs_mean.msl/100).plot.contour(ax=ax,
                    colors = 'k',
                    transform=ccrs.PlateCarree(),
                    levels=np.arange(900,1100,4),
                    add_colorbar=False,
                    linewidths=0.5)
divider = make_axes_locatable(ax)
ax_cb = divider.new_horizontal(size="5%", pad=0.001, axes_class=plt.Axes, aspect=30)
fig.add_axes(ax_cb)
cb = plt.colorbar(c, cax=ax_cb)
cb.set_label(label='Sigma',fontsize=16)
plt.show()

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection=ccrs.Miller())
ax.add_feature(cf.NaturalEarthFeature(
    'cultural', 'admin_1_states_provinces_lines', '50m',
    edgecolor='gray', facecolor='none'))
ax.add_feature(cf.NaturalEarthFeature(
    'cultural', 'admin_1_states_provinces_lines', '50m',
    edgecolor='gray', facecolor='none'))        
ax.add_feature(cf.LAKES, facecolor='gray')
ax.add_feature(cf.BORDERS, edgecolor='gray')
ax.add_feature(cf.RIVERS, edgecolor='gray')
ax.add_feature(cf.OCEAN, facecolor='gray')
ax.set_extent([-180,-50,20,60],crs=ccrs.PlateCarree())
ax.coastlines(resolution='50m')
subset_vals_fix = gefs_sprd
subset_vals_fix['longitude'] = (subset_vals_fix['longitude']  + 180) % 360 - 180
subset_vals_fix  = subset_vals_fix.sortby(subset_vals_fix['longitude'] )
c = subset_vals_fix.msl.plot.imshow(ax=ax,
                             transform=ccrs.PlateCarree(),
                                add_colorbar=False,cmap='viridis')
(gefs_mean.msl/100).plot.contour(ax=ax,
                    colors = 'k',
                    transform=ccrs.PlateCarree(),
                    levels=np.arange(900,1100,4),
                    add_colorbar=False,
                    linewidths=0.5)
# divider = make_axes_locatable(ax)
# ax_cb = divider.new_horizontal(size="5%", pad=0.001, axes_class=plt.Axes, aspect=30)
# fig.add_axes(ax_cb)
# cb = plt.colorbar(c, cax=ax_cb)
# cb.set_label(label='Sigma',fontsize=16)
plt.show()